In [1]:
import scipy.stats.distributions as dist
from astropy.table import Table
import plotting_func

from scipy.stats import binned_statistic, scoreatpercentile

#-------------------------------------------------------
import matplotlib as mpl
from prefig import Prefig
Prefig()
% matplotlib inline

from matplotlib.ticker import MaxNLocator

plt.rcParams['font.family'] = 'serif'
plt.rcParams['figure.figsize'] = (10.0, 8)
plt.rcParams['font.size'] = 18
mpl.ticker.AutoLocator.default_params['nbins'] = 6
mpl.ticker.AutoLocator.default_params['prune'] = 'both'
#------------------------------------------------------

source_dir = '../../fits/'
data_file = 'volume_limited_sample.fits'
debiased_file = 'debiased_volume_limited_sample.fits'

ImportError: No module named 'plotting_func'

In [ ]:
def get_column(c,data,col_name):
  
    if len(c) == 2:
        column = data.field(c[0]) - data.field(c[1])
    else:
        column = data.field(c[0])
        if len(c) > 2:
            print(col_name + "has >2 fields. Only the first field was used.")
  
    return column

def load(cx,p_th=0.5,N_th=5,mass_lim=False):

    gal_data = fits.getdata(source_dir + data_file,1) 
    debiased_values = fits.getdata(source_dir + debiased_file,1)
            
    x_column = get_column(c=cx,data=gal_data,col_name="x") 
    
    f_v = np.array([debiased_values.field(c) 
                    for c in debiased_values.columns.names]).T
    
    table = np.concatenate([f_v,np.array([x_column]).T],axis=1)
    
    table = Table(table,names=('p_1','p_2','p_3','p_4','p_5','p_ct','x'))
        
    p_spiral = (
        gal_data.field("t01_smooth_or_features_a02_features_or_disk_debiased")
        *gal_data.field("t02_edgeon_a05_no_debiased")
        *gal_data.field("t04_spiral_a08_spiral_debiased"))
    N_spiral = (gal_data.field("t04_spiral_a08_spiral_count")
                -gal_data.field('t11_arms_number_a37_cant_tell_count')) # Load values to
    # allow data cuts to be made. Keep only gals where more than 5 people classified the 
    # the spiral arm number.
    
    spiral_select = (p_spiral > p_th) & (N_spiral >= N_th)
    finite_select = (np.isfinite(table['x']))
    
    if mass_lim == True:
        mass_cut = gal_data.field('LOGMSTAR_BALDRY06') > 10.6
        full_table = table[finite_select & mass_cut]
        spiral_table = table[finite_select & spiral_select & mass_cut]
        
    else:
        full_table = table[finite_select]
        spiral_table = table[finite_select & spiral_select]
        
    return full_table,spiral_table

In [ ]:
def assign(table,th=0,redistribute=False,rd_th=0,ct_th=0,print_sizes=False):

    m_columns = ['p_1','p_2','p_3','p_4','p_5','p_ct']
    
    m_array = np.array([table[column] for column in m_columns]).T
    
    arm_assignments = np.ones(len(table))*(-999) # Assigned arm numbers 
    # initially is an array of -999s. -999 means 'no assignment'.
    for m in range(6):
        a = (np.argmax(m_array,axis=1) == m) & (m_array[:,m] >= th)
        arm_assignments[a] = m
        
    if redistribute is True: # Redistribute according to thresholds.
        for m in range(5):
            arm_assignments[(np.argmax(m_array[:,:5],axis=1) == m) 
                & (arm_assignments == 5) & (m_array[:,m]/m_array[:,5] > rd_th) 
                & (m_array[:,5] <= ct_th)] = m

    if print_sizes is True:
        print("total sample: " + str(len(arm_assignments)))
        print("total 'assigned' sample: " 
              + str(np.sum(arm_assignments != -999)))
        for m in range(6):
            print("m = " + str(m+1) + ": " 
                  + str(np.sum(arm_assignments == m)))
      
    return arm_assignments

In [ ]:
def bin_by_column(column, nbins, fixedcount=True):
    sorted_indices = np.argsort(column)
    if fixedcount:
        bin_edges = np.linspace(0, 1, nbins + 1)
        bin_edges[-1] += 1
        values = np.empty(len(column))
        values[sorted_indices] = np.linspace(0, 1, len(column))
        bins = np.digitize(values, bins=bin_edges)
    else:
        bin_edges = np.linspace(np.min(column),np.max(column), nbins + 1)
        bin_edges[-1] += 1
        values = column
        bins = np.digitize(values, bins=bin_edges)
    x, b, n = binned_statistic(values, column, bins=bin_edges)
    return x, bins

In [ ]:
def binned_morph_frac(morph, bins):
    total = np.bincount(bins)[1:]
    frac = []
    for m in np.unique(morph):
        count = np.bincount(bins[morph == m], minlength=bins.max()+1)[1:]
        frac.append(frac_errors(count, total))
    return frac

In [ ]:
def plot_morph_frac_trend(x, frac, frac_raw=None, xlabel='redshift', answerlabels=None,label_position=[0.04,0.95]):
    C = ['orange','red','magenta','green','blue']
    nmorph = len(frac)
    fig, axarr = plt.subplots(5, 1, sharex=True, sharey=False, figsize=(10, 3*nmorph))
    
    for m in range(5):
        ax = axarr[m]
        ax.yaxis.set_major_locator(mpl.ticker.MaxNLocator(nbins=4, prune="upper"))
        if frac_raw is not None:
            ax.plot(x, frac_raw[m]['frac'], '--', color=C[m], lw=2, alpha=0.5)
            ax.fill_between(x, frac_raw[m]['low'], frac_raw[m]['high'], color=C[m], alpha=0.25)
        mean = frac[m]['frac'].mean()
        ax.hlines([mean], -100, 100, linestyle=':', color=C[m], lw=3)
        ax.plot(x, frac[m]['frac'], color=C[m], lw=2)
        ax.fill_between(x, frac[m]['low'], frac[m]['high'], color=C[m], alpha=0.5)
        
        ax.set_ylabel('fraction',fontsize=18)
        if answerlabels is not None:
            ax.text(label_position[0], label_position[1], answerlabels[m], transform=ax.transAxes, va='top',fontsize=20)
            
            ax.yaxis.set_major_locator(MaxNLocator(nbins=6,prune='both'))
    ax.set_xlabel(xlabel,fontsize=18)
    
    plt.subplots_adjust(hspace=0)

In [ ]:
def frac_errors(k, n, c=0.683):
    # Gets the errors according to the Cameron et al. paper.
    
    # cope with both array and individual value inputs
    k = np.array(k, np.float, ndmin=1)
    frac = k / n
    low = dist.beta.ppf((1-c)/2., k+1, n-k+1)
    high = dist.beta.ppf(1-(1-c)/2., k+1, n-k+1)    
    # ensure confidence intervals are sensible for
    # extreme fractions
    low = np.where(frac < low, 0.0, low)
    high = np.where(frac > high, 1.0, high)
    # return sensible shaped arrays
    out = Table((frac, low, high), names=('frac', 'low', 'high'))
    return out

In [ ]:
clr = ['orange','red','magenta','green','blue']

a = ['$m=1$','$m=2$','$m=3$','$m=4$','$m=5+$']

In [ ]:
# Stellar mass ----

data,spirals = load(cx=['LOGMSTAR_BALDRY06'],mass_lim=True)
arm_assignments = assign(table=spirals,th=0,redistribute=True,rd_th=0,ct_th=10)
bin_values,bin_assignments = bin_by_column(spirals['x'],20)

frac = binned_morph_frac(arm_assignments, bin_assignments)
plot_morph_frac_trend(bin_values,frac,xlabel='$\log(M/M_{\odot})}$',answerlabels=a)

plt.xlim(10.6,11.22)
    
plt.savefig('../../output_images/fractional_plots/mass_plot.pdf')

In [ ]:
# Colour ----

data,spirals = load(cx=['PETROMAG_MG','PETROMAG_MR'],mass_lim=True)
arm_assignments = assign(table=spirals,th=0,redistribute=True,rd_th=0,ct_th=10)
bin_values,bin_assignments = bin_by_column(spirals['x'],20)

frac = binned_morph_frac(arm_assignments, bin_assignments)
plot_morph_frac_trend(bin_values,frac,xlabel='$g-r$',answerlabels=a,label_position=[0.04,0.95])

plt.xlim(0.4,0.8)
    
plt.savefig('../../output_images/fractional_plots/colour_plot.pdf')

In [ ]:
#print(bin_values)
#print(frac[1])
#print(frac[4])

full_b = spirals[bin_assignments == 1]
full_r = spirals[bin_assignments == 20]

many_select = (arm_assignments == 2) + (arm_assignments == 3) + (arm_assignments == 4)

a_b = spirals[(bin_assignments == 1) & (many_select == 1)]
a_r = spirals[(bin_assignments == 20) & (many_select == 1)]

c = 0.683

def get_lh(n,k):
    
    frac = k/n

    low = dist.beta.ppf((1-c)/2., k+1, n-k+1)
    high = dist.beta.ppf(1-(1-c)/2., k+1, n-k+1)
    
    print(frac,low,high)
    
    return None

get_lh(len(full_b),len(a_b))
get_lh(len(full_r),len(a_r))

In [ ]:
fig,ax = plt.subplots()

means = np.zeros((5,3))

means[:,0]=np.arange(5) + 1

means[:,1] = [np.mean(spirals['x'][arm_assignments == m]) for m in range(5)]
means[:,2] = [np.std(spirals['x'][arm_assignments == m]) for m in range(5)]

plt.errorbar(means[:,0],means[:,1],yerr=means[:,2],fmt='ko')

print(np.std(spirals['x']))

plt.xlim(0,6)
plt.xticks([1,2,3,4,5])
plt.ylim(0.425,0.775)
plt.yticks(np.arange(0.5,0.79,0.1))

#labels = [item.get_text() for item in ax.get_xticklabels()]
labels = ['1','2','3','4','5+']
ax.set_xticklabels(labels)

plt.xlabel('$m$')
plt.ylabel('$g-r$')

In [ ]:
# SFR ----

data,spirals = load(cx=['ssfr_total_avg'],mass_lim=True)
arm_assignments = assign(table=spirals,th=0,redistribute=True,rd_th=0,ct_th=10)
bin_values,bin_assignments = bin_by_column(spirals['x'],20)

frac = binned_morph_frac(arm_assignments, bin_assignments)
plot_morph_frac_trend(bin_values,frac,xlabel='$sSFR [\log(M_{\odot} /M_{*}yr)]$',answerlabels=a)

plt.xlim(-12.25,-9.6)
    
plt.savefig('../../output_images/fractional_plots/ssfr_plot.pdf')

In [ ]:
data,spirals = load(cx=['IVAN_DENSITY'],mass_lim=True)
arm_assignments = assign(table=spirals,th=0,redistribute=True,rd_th=0,ct_th=10)
bin_values,bin_assignments = bin_by_column(spirals['x'],20)

frac = binned_morph_frac(arm_assignments, bin_assignments)
plot_morph_frac_trend(bin_values,frac,xlabel='$\log \Sigma$',answerlabels=a)

plt.xlim(-1.25,1.25)
    
plt.savefig('../../output_images/fractional_plots/density.pdf')

In [ ]:
print(bin_values)
print(frac[1])
print(frac[4])